# Heatmaps elaborated with R showing where La Liga teams execute defensive play (selection of matches from season 2020-2021). 
**Analyses show dramatic differences among teams that apply pressure on the offensive field, and those choosing to defend around their own boxes.**

Analysis inspired by the document 'Accessing & Working With StatsBomb Data In R'

Data from StatsBomb: https://github.com/statsbomb/open-data/tree/master/data 

See their main web page: https://statsbomb.com/

#Football #Soccer #FootStats #LaLiga #RStats

In [ ]:
#Install the following packages if you need to...
install.packages("devtools")
install.packages("remotes")
remotes::install_version("SDMTools", "1.1-221")
devtools::install_github("statsbomb/StatsBombR")
devtools::install_github("FCrSTATS/SBpitch")

In [ ]:
#Load the following libraries
library(StatsBombR)
library(tidyr)
library(dplyr)
library(ggplot2)
library(SBpitch)
library(forcats)
library(grid)

#Grab the data, we will choose only the available matches from La Liga, season 2020-21
LaLiga <- FreeCompetitions() %>% filter(competition_id== 11 & season_id== 90)
Matches_sp <- FreeMatches(LaLiga)
Events_LL <- free_allevents(MatchesDF = Matches_sp, Parallel = T)
Events_LL <- allclean(Events_LL)

#We correct some locations in the data, since some may be beyond the pitch limits
heatmap <- Events_LL %>% mutate(location.x= ifelse(location.x>120, 120, location.x), location.y= ifelse(location.y>80, 80, location.y), location.x= ifelse(location.x<0, 0, location.x), location.y= ifelse(location.y<0, 0, location.y))

#We create bins for the x and y axis
heatmap$x.bin <- cut(heatmap$location.x, breaks = seq(0, 120, 20), include.lowest = TRUE)                    
heatmap$y.bin <- cut(heatmap$location.y, breaks = seq(0, 80, 20), include.lowest = TRUE)                    

#We select the most representative defensive plays, namely, applying pressure, tackling, commiting a foul, intercepting the ball, and blocking it.
#Then we group the data and conduct the calculations necessary for the heatmaps (how much more or less defensive play each team applies in each bin of the pich compared to the average of all teams)
heatmap <- heatmap %>% filter(type.name=="Pressure" | duel.type.name=="Tackle" | type.name=="Foul Committed" | type.name=="Interception" | type.name=="Block") %>% group_by(team.name) %>% mutate(total_DA= n()) %>% group_by(team.name, x.bin, y.bin) %>% summarise(total_DA= max(total_DA), bin_DA= n(), bin_pct= bin_DA*100/ total_DA, location.x= median(location.x), location.y= median(location.y)) %>% group_by(x.bin, y.bin) %>% mutate(league_ave= mean(bin_pct)) %>% group_by(team.name, x.bin, y.bin) %>% mutate(diffs= bin_pct- league_ave)

#We prepare the colors to be displayed
defensiveactivitycolors <- c("#dc2429", "#dc2329", "#df272d", "#df3238", "#e14348", "#e44d51", "#e35256", "#e76266", "#e9777b", "#ec8589", "#ec898d", "#ef9195", "#ef9ea1", "#f0a6a9", "#f2abae", "#f4b9bc", "#f8d1d2", "#f9e0e2", "#f7e1e3", "#f5e2e4", "#d4d5d8", "#d1d3d8", "#cdd2d6", "#c8cdd3", "#c0c7cd", "#b9c0c8", "#b5bcc3", "#909ba5", "#8f9aa5", "#818c98", "#798590", "#697785", "#526173", "#435367", "#3a4b60", "#2e4257", "#1d3048", "#11263e", "#11273e", "#0d233a", "#020c16")

#We set the plotting area
options(repr.plot.width=15, repr.plot.height=8)

#We create a function that will display the heatmap for every team
heatmap_DA <- function(team){
plot <- ggplot(data= filter(heatmap, team.name== team), aes(x = location.x, y = location.y, fill = diffs, group =diffs)) +
  geom_bin2d(binwidth = c(20, 20), position = "identity", alpha = 0.9)+
annotate("rect",xmin = 0, xmax = 120, ymin = 0, ymax = 80, fill = NA, colour = "black", size = 0.6) +
  annotate("rect",xmin = 0, xmax = 60, ymin = 0, ymax = 80, fill = NA, colour = "black", size = 0.6) +
  annotate("rect",xmin = 18, xmax = 0, ymin = 18, ymax = 62, fill = NA, colour = "white", size = 0.6) +
  annotate("rect",xmin = 102, xmax = 120, ymin = 18, ymax = 62, fill = NA, colour = "white", size = 0.6) +
  annotate("rect",xmin = 0, xmax = 6, ymin = 30, ymax = 50, fill = NA, colour = "white", size = 0.6) +
  annotate("rect",xmin = 120, xmax = 114, ymin = 30, ymax = 50, fill = NA, colour = "white", size = 0.6) +
  annotate("rect",xmin = 120, xmax = 120.5, ymin =36, ymax = 44, fill = NA, colour = "black", size = 0.6) +
  annotate("rect",xmin = 0, xmax = -0.5, ymin =36, ymax = 44, fill = NA, colour = "black", size = 0.6) +
  annotate("segment", x = 60, xend = 60, y = -0.5, yend = 80.5, colour = "white", size = 0.6)+
  annotate("segment", x = 0, xend = 0, y = 0, yend = 80, colour = "black", size = 0.6)+
  annotate("segment", x = 120, xend = 120, y = 0, yend = 80, colour = "black", size = 0.6)+
theme(rect = element_blank(),
      line = element_blank()) +
  annotate("point", x = 12 , y = 40, colour = "white", size = 1.05) +
  annotate("point", x = 108 , y = 40, colour = "white", size = 1.05) +
  annotate("path", colour = "white", size = 0.6,
           x=60+10*cos(seq(0,2*pi,length.out=2000)),
           y=40+10*sin(seq(0,2*pi,length.out=2000)))+
  annotate("point", x = 60 , y = 40, colour = "white", size = 1.05) +
  annotate("path", x=12+10*cos(seq(-0.3*pi,0.3*pi,length.out=30)), size = 0.6,
           y=40+10*sin(seq(-0.3*pi,0.3*pi,length.out=30)), col="white") +
  annotate("path", x=108-10*cos(seq(-0.3*pi,0.3*pi,length.out=30)), size = 0.6,
           y=40-10*sin(seq(-0.3*pi,0.3*pi,length.out=30)), col="white")+
  theme(axis.text.x=element_blank(),
        axis.title.x = element_blank(),
        axis.title.y = element_blank(),
        plot.title = element_text(size=22))+
  scale_y_reverse()+
  scale_fill_gradientn(colours = defensiveactivitycolors, trans = "reverse")+
  labs(title = paste("Defensive play across the pitch by", team)) +
  coord_fixed(95/100)+
    annotation_custom(grob = linesGrob(arrow=arrow(type="open", ends="last", length=unit(2.55,"mm")), gp=gpar(col="black", fill=NA, lwd=2.2)), xmin=25, xmax = 95, ymin = -83, ymax = -83)+
    guides(fill = guide_legend(reverse = TRUE))

return(plot)
}

#Finally, we plot the function for all the teams from La Liga
teams= unique(heatmap$team.name)
for (team in teams){
print(heatmap_DA(team))
}